# TVCG and CG&A Papers presented at VIS

Here we create a new sheet on vispubdata that contains articles accepted directly to the IEEE TVCG journal and the IEEE CG&A magazine. These articles were presented at VIS but were not reviewed at VIS. Instead, they were reviewed by the regular journal/magazine review process.

The list of journal papers is kept up-to-date by Tobias Isenberg in this location: https://docs.google.com/spreadsheets/d/1I6n4a6xvmoanAIDiSsGlaOVljAJ5IkT2C_naI-dStNo/

If you have new journal articles you would like to add that are not yet in this spreadsheet then, copy the spreadsheet, add the dois of the new articles, then change the url below to your new spreadsheet (or read from a csv file).

Run this code after you extracted data from the DBLP (see its respective subfolder)

In [70]:
#First load what we need to load

import pandas as pd
import urllib.request, json 
import requests
import os
import csv
import re
from crossref.restful import Works, Etiquette

with open('ieeexplore-apikey.txt') as f:
    apikey = f.readline()
    
youremail = "petra.isenberg@inria.fr" #replace this with your own email address. This is required for querying CrossRef
    

In [3]:
journals_sheet_url = "https://docs.google.com/spreadsheets/d/1I6n4a6xvmoanAIDiSsGlaOVljAJ5IkT2C_naI-dStNo/gviz/tq?tqx=out:csv"
journals_df = pd.read_csv(journals_sheet_url, keep_default_na=False)
journals_df.head()

,year,journal,title,doi,doi link,xplore id,ieee xplore direct link,ieee xplore pdf direct link,pdf filename,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,2011,TVCG,Exploring Brain Connectivity with Two-Dimensio...,10.1109/TVCG.2011.82,https://doi.org/10.1109/TVCG.2011.82,5753898,https://ieeexplore.ieee.org/document/5753898,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,05753898.pdf,,...,,,,,,,,,,
1,2011,TVCG,Hierarchical Line Integration,10.1109/TVCG.2010.227,https://doi.org/10.1109/TVCG.2010.227,5611509,https://ieeexplore.ieee.org/document/5611509,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,05611509.pdf,,...,,,,,,,,,,
2,2011,TVCG,Streamline Integration Using MPI-Hybrid Parall...,10.1109/TVCG.2010.259,https://doi.org/10.1109/TVCG.2010.259,5669297,https://ieeexplore.ieee.org/document/5669297,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,05669297.pdf,,...,,,,,,,,,,
3,2011,TVCG,Efficient Visibility Encoding for Dynamic Illu...,10.1109/TVCG.2011.35,https://doi.org/10.1109/TVCG.2011.35,5710905,https://ieeexplore.ieee.org/document/5710905,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,05710905.pdf,,...,,,,,,,,,,
4,2011,TVCG,Morse Set Classification and Hierarchical Refi...,10.1109/TVCG.2011.107,https://doi.org/10.1109/TVCG.2011.107,5928334,https://ieeexplore.ieee.org/document/5928334,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,05928334.pdf,,...,,,,,,,,,,


## Download data from IEEEXplore

Make sure you have an API key first (and loaded above)



In [8]:
baseurl =  "https://ieeexploreapi.ieee.org/api/v1/search/articles?parameter&apikey="+apikey+"&max_records=200&doi="

#https://ieeexploreapi.ieee.org/api/v1/search/articles?querytext=(rfid%20OR%20%22internet%20of%20things%22)&apikey=

In [11]:
#try to generate a search string with 200 DOIs at a time

# Function to concatenate DOIs in blocks of 200 with "%20OR%20%". 
def concatenate_dois(dois, block_size=200, separator="%20OR%20%"):
    concatenated_list = []
    for i in range(0, len(dois), block_size):
        block = dois[i:i + block_size]
        concatenated_string = separator.join(block)
        concatenated_list.append(concatenated_string)
    return concatenated_list

# Concatenate the DOIs
concatenated_dois = concatenate_dois(journals_df["doi"].tolist())

# ask IEEEXplore and then save the results
# we have to do this blocking because there is a rate limit that is lower than the number of articles we have

i = 0
for block in concatenated_dois:
    fullurl = baseurl + block
    full_filename = "journal-data/original-data/ieeexplore-article-data-"+str(i)+".json"
    i = i + 1
    with open(full_filename, 'w+', encoding="utf-8") as f:
            resp = requests.get(fullurl, verify=True)
            f.write(resp.text)
    


## Convert the data to vispubdata format

Similar to how it's done for the vis papers ...
Here we convert the .json file into something more flat and similar to vispubdata. Run this code if you updated the .json files above.

This code comes from here:
https://github.com/vinay20045/json-to-csv/blob/master/json_to_csv.py



In [20]:
##
# Convert to string keeping encoding in mind...
##
def to_string(s):
    
    try:
        return str(s)
    except:
        #Change the encoding type if needed
        return s.encode('utf-8')

    
def reduce_item(key, value):
    global reduced_item
    
    #Reduction Condition 1
    if type(value) is list:
        i=0
        for sub_item in value:
            reduce_item(key+'_'+to_string(i), sub_item)
            i=i+1

    #Reduction Condition 2
    elif type(value) is dict:
        sub_keys = value.keys()
        for sub_key in sub_keys:
            reduce_item(key+'_'+to_string(sub_key), value[sub_key])
    
    #Base Condition
    else:
        reduced_item[to_string(key)] = to_string(value)

folderdir = 'journal-data/original-data'
generated_data_path = 'journal-data/generated-data'
 
# giving file extension
ext = ('.json')


 
# iterating over all files
# iterating over directory and subdirectory to get desired result
for path, dirc, files in os.walk(folderdir):
    for name in files:
        if name.endswith(ext):
            csv_file_path = generated_data_path+"/"+name+".csv"
            node = "articles"
        
            pathExists = os.path.exists(generated_data_path)
            if not pathExists:
                os.makedirs(generated_data_path)
        
            json_file_path = path + "/" + name
            fp = open(json_file_path, mode = 'rb')
            json_value = fp.read()
            raw_data = json.loads(json_value)        
        
            print(json_file_path)  # printing file name of desired extension
            
            try:
                data_to_be_processed = raw_data[node]
            except:
                data_to_be_processed = raw_data

            processed_data = []
            header = []
            for item in data_to_be_processed:
                reduced_item = {}
                reduce_item(node, item)

                header += reduced_item.keys()

                processed_data.append(reduced_item)

            header = list(set(header))
            header.sort()

            with open(csv_file_path, 'w+',encoding="utf8") as f:
                writer = csv.DictWriter(f, header, dialect='excel')
                writer.writeheader()
                for row in processed_data:
                    writer.writerow(row)

        #print ("Just completed writing "+csv_file_path+" file with %d columns" % len(header))

        else:
            continue


journal-data/original-data/ieeexplore-article-data-0.json
journal-data/original-data/ieeexplore-article-data-1.json
journal-data/original-data/ieeexplore-article-data-2.json


## Convert IEEEXplore csvs to Vispubdata

Here we convert the data from the .csv files into the vispubdata format


#### Read the latest vispubdata

In [22]:


# Google Sheet URL
vispubdata_sheet_url = "https://docs.google.com/spreadsheets/d/1xgoOPu28dQSSGPIp_HHQs0uvvcyLNdkMF9XtRajhhxU/gviz/tq?gid=1573772404&tqx=out:csv"

# Read the CSV into a DataFrame
vispubdata = pd.read_csv(vispubdata_sheet_url, keep_default_na=False)

print(vispubdata.columns)

#if you ever want to introduce new columns you could try to do it here. I usually add them on the Google sheet. 
final_columns = vispubdata.columns
#final_columns = ['Conference', 'Year', 'Title', 'DOI', 'Link', 'FirstPage','LastPage','PaperType','Abstract','AuthorNames-Deduped','AuthorNames','AuthorAffiliation','InternalReferences','AuthorKeywords','AminerCitationCount','CitationCount_CrossRef','PubsCited_CrossRef',' Downloads_Xplore','Award','GraphicsReplicabilityStamp']

#double check that the names are correct here
crossRefCitation_column = "CitationCount_CrossRef"
crossRefPubsCited_column = 'PubsCited_CrossRef'
downloads_column = "Downloads_Xplore"



Index(['Journal', 'Year', 'Title', 'DOI', 'Link', 'FirstPage', 'LastPage',
       'PaperType', 'Abstract', 'AuthorNames-Deduped', 'AuthorNames',
       'AuthorAffiliation', 'InternalReferences', 'AuthorKeywords',
       'AminerCitationCount', 'CitationCount_CrossRef', 'PubsCited_CrossRef',
       'Downloads_Xplore', 'Award', 'GraphicsReplicabilityStamp'],
      dtype='object')


#### Helper methods

In [26]:
#we need this later to sort the author columns by the number hidden in its name  
def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]

In [39]:
#Here we prepare the data structure that will resemble the final vispubdata table
def prepareXploreDFTable(xplore_df):
    
    #get all column names
    columns = xplore_df.columns

    #remove all the columns we don't need
    #-------------------------------------------------------------
    xplore_df.drop('articles_access_type', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_abstract_url', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_publisher', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_article_number', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_volume', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_rank', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_publication_number', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_publication_date', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_pdf_url', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_partnum', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_issue', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_issn', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_is_number', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_html_url', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_publication_title', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_citing_patent_count', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_conference_location', axis=1, inplace=True,errors='ignore')
    xplore_df.drop('articles_conference_dates', axis=1, inplace=True,errors='ignore')
 

    #WARNING: I assume the file is correctly ordered. If not we need to do something more fancy
    columns_to_drop = columns[columns.str.contains("author_order")] 
    xplore_df.drop(columns_to_drop, axis=1, inplace=True)

    columns_to_drop = columns[columns.str.contains("ieee_terms")] 
    xplore_df.drop(columns_to_drop, axis=1, inplace=True)
    
    columns_to_drop = columns[columns.str.contains("_id")] 
    xplore_df.drop(columns_to_drop, axis=1, inplace=True)
    
    columns_to_drop = columns[columns.str.contains("authorUrl")] 
    xplore_df.drop(columns_to_drop, axis=1, inplace=True)
    
    columns_to_drop = columns[columns.str.contains("isbn")] 
    xplore_df.drop(columns_to_drop, axis=1, inplace=True)
    

    #rename the columns we want to keep
    xplore_df.rename(index=str, inplace=True, columns={"articles_title":"Title",
                                                       "articles_start_page":"FirstPage",
                                                       "articles_abstract":"Abstract",
                                                       "articles_publication_year": "Year",
                                                       "articles_content_type": "PaperType",
                                                       "articles_end_page":"LastPage",
                                                       "articles_doi":"DOI",
                                                       "articles_citing_paper_count":"CitationCount_CrossRef",
                                                       "articles_download_count":"Downloads_Xplore"})


    #put all author full names together
    #----------------------------------------
    author_columns = columns[columns.str.contains("full_name")].tolist()
    author_columns.sort(key=num_sort)
    
    xplore_df[author_columns].fillna(value="")
    authors = xplore_df[author_columns].apply(lambda x: ';'.join(x.dropna().values.tolist()), axis=1)
    authors = authors.str.rstrip(";")
    #we have the authors put together, now add them to the df
    xplore_df['AuthorNames'] = authors
    #now remove all the individual columns that we no longer need
    xplore_df.drop(author_columns, axis=1, inplace=True)

    #put all affiliations together
    #------------------------------------------
    ##We have to do something more complicated for the affiliations because the csv file does not contain an affiliation column if none of the authors in a given position has an affiliation
    
    #Careful. IEEEXplore seems to change the way it handles the affiliations. Their json changes each year. 
    #Also, the original json can now handle multiple affiliations - todo for the future. Requires an update to vispubdata.
    
    xplore_df['AuthorAffiliation'] = "" #make an empty column first
    xplore_df["AuthorCount"] =  xplore_df['AuthorNames'].str.count(';') + 1
    
    for index, row in xplore_df.iterrows():
        #we need to find out how many authors a paper has first
        authorCount = row["AuthorCount"]
        affiliations = ""
        for i in range(0,authorCount):
            author_column_name = author_columns[i]
            affcolumn = author_column_name.replace("full_name","affiliation")
            if affcolumn in xplore_df.columns:
                #we're doing this a few too many times here but we don't care for speed, yet
                affiliations = affiliations + ";" + row[affcolumn]
            else:
                affiliations = affiliations + ";" + ""
        affiliations = affiliations[1:]
        #print(affiliations)
        xplore_df.at[index,'AuthorAffiliation'] = affiliations
    
    
    xplore_df.drop(["AuthorCount"], axis=1, inplace=True)
    
    affiliation_columns = columns[columns.str.contains("affiliation")]
    #now remove all the individual columns that we no longer need
    xplore_df.drop(affiliation_columns, axis=1, inplace=True)

    #put all author keywords together
    #------------------------------------------
    kw_columns = columns[columns.str.contains("author_terms")]
    xplore_df[kw_columns].fillna(value="")
    keywords = xplore_df[kw_columns].apply(lambda x: ','.join(x.dropna().values.tolist()), axis=1)
    keywords = keywords.apply(lambda x: x.rstrip(','))

    #we have the keywords put together, now add them to the df
    xplore_df['AuthorKeywords'] = keywords
    #now remove all the individual columns that we no longer need
    xplore_df.drop(kw_columns, axis=1, inplace=True)

    #create the link column
    #--------------------------------------------
    xplore_df["Link"] = 'http://dx.doi.org/' + xplore_df["DOI"]

    #now add columns that are missing
    xplore_df_columns = xplore_df.columns
    
    for c in final_columns:
        if not c in xplore_df_columns:
            if c == "Conference":
                xplore_df["Conference"] = ["conference_external"] * len(xplore_df.index)
            else:
                xplore_df[c] = [""] * len(xplore_df.index)
    
    for c in xplore_df_columns:
        if not c in final_columns:
            #we remove all columns that we haven't captured yet
            xplore_df.drop([c], axis=1, inplace=True)
            
    #now reorder the columns
    xplore_df = xplore_df[final_columns]


In [40]:
def checkJournalTitle(xplore_df):
    #DOIs are not case_sensitive!
    journals_df['doi'] = journals_df['doi'].str.lower()
    xplore_df['DOI'] = xplore_df['DOI'].str.lower()
    
    for index, row in xplore_df.iterrows():
        xplore_doi = row['DOI']
        journal_name = journals_df.loc[journals_df['doi'] == xplore_doi,"journal"].tolist()[0]
        xplore_df.loc[index,'Journal'] = journal_name
        
    
    
                

In [41]:
def convertFile(csv_filename):
    path = "journal-data/generated-data/"+csv_filename
    
    #double-checking that the file exists
    if(os.path.isfile(path)):
        print("Your file " + path + " exists")
    else:
        print("Your file " + path + " does not exist")
        
    #now we load it
    xplore_df = pd.read_csv(path,dtype=object, encoding ='utf-8',keep_default_na=False,dialect='excel')
    #now prepare it
    prepareXploreDFTable(xplore_df)
    
    checkJournalTitle(xplore_df)
    
    return xplore_df
    

In [60]:
folderdir = 'journal-data/generated-data'
 
ext = ('.csv')
xploredfs = []

for path, dirc, files in os.walk(folderdir):
    for name in files:
        if "ieeexplore" in name:
            xdf = convertFile(name)
            xploredfs.append(xdf)

ieeexplore_vispub_df =  pd.concat(xploredfs, ignore_index=True)
#full_xplore_df.to_csv("results/Vispubdata-journals.csv",index =False)
#print("File saved")

Your file journal-data/generated-data/ieeexplore-article-data-0.json.csv exists
Your file journal-data/generated-data/ieeexplore-article-data-1.json.csv exists
Your file journal-data/generated-data/ieeexplore-article-data-2.json.csv exists


#### Now fill missing entries and fix some others

##### Fixing the Publication Type

- TVCG papers are Journals, so they will get a "J" label
- CG&A papers are magazine articles, they will get a "MAG" label

In [61]:
ieeexplore_vispub_df.loc[ieeexplore_vispub_df["Journal"] == "TVCG","PaperType"] = "J"
ieeexplore_vispub_df.loc[ieeexplore_vispub_df["Journal"] == "CG&A","PaperType"] = "MAG"
ieeexplore_vispub_df.head()

,Abstract,CitationCount_CrossRef,PaperType,DOI,Downloads_Xplore,LastPage,Year,FirstPage,Title,AuthorNames,AuthorAffiliation,AuthorKeywords,Link,Journal,AuthorNames-Deduped,InternalReferences,AminerCitationCount,PubsCited_CrossRef,Award,GraphicsReplicabilityStamp
0,"Identifying, tracking and understanding change...",131,J,10.1109/tvcg.2013.254,1647,754,2014,740,GraphDiaries: Animated Transitions andTemporal...,Benjamin Bach;Emmanuel Pietriga;Jean-Daniel Fe...,"INRIA, Saclay, France;INRIA, Saclay, France an...","Dynamic networks,graph visualization,,,,,,,,,,...",http://dx.doi.org/10.1109/TVCG.2013.254,TVCG,,,,,,
1,This paper presents an acceleration scheme for...,49,J,10.1109/tvcg.2010.227,614,1163,2011,1148,Hierarchical Line Integration,Marcel Hlawatsch;Filip Sadlo;Daniel Weiskopf,Visualisierungsinstitut der Universität Stuttg...,"Flow visualization,integral curves,,,,,,,,,,,h...",http://dx.doi.org/10.1109/TVCG.2010.227,TVCG,,,,,,
2,This article explores the potential for creati...,11,MAG,10.1109/mcg.2015.71,1134,81,2015,73,Visualizing Personal Progress in Participatory...,Jo Wood,"City University, London","computer graphics,personal visualization,,,,,,...",http://dx.doi.org/10.1109/MCG.2015.71,CG&A,,,,,,
3,Feature Flow Fields are a well-accepted approa...,48,J,10.1109/tvcg.2010.93,775,780,2011,770,Stable Feature Flow Fields,Tino Weinkauf;Holger Theisel;Allen Van Gelder;...,"Learning and Graphics Group, Courant Institute...","Flow visualization,feature extraction.",http://dx.doi.org/10.1109/TVCG.2010.93,TVCG,,,,,,
4,In uncertain scalar fields where data values v...,29,J,10.1109/tvcg.2013.92,581,1961,2013,1948,Visualizing the Variability of Gradients in Un...,Tobias Pfaffelmoser;Mihaela Mihai;Rüdiger West...,"Computer Graphics and Visualization Group, Tec...","Uncertainty visualization,gradient variability...",http://dx.doi.org/10.1109/TVCG.2013.92,TVCG,,,,,,


In [62]:
 ieeexplore_vispub_df.to_csv("results/DEBUG_ieeexplore_journals_df.csv") #some debugging output if you'd like to check that so far things are in order

### Merging with the current vispubdata journal spreadsheet

Now we merge the current vispubdata and the new table.
We want to keep our current version of vispubdata and add the new papers, then we need to copy over the new citation and download counts

In [63]:
vispubdata #the current version of vispubdata journal articles (loaded above)
#we probably already did this but just in case

ieeexplore_vispub_df #all vis papers with data from IEEEXplore -> that is, some papers will be missing but there will be some new ones in here
ieeexplore_vispub_df['DOI'] = ieeexplore_vispub_df['DOI'].str.lower()
vispubdata['DOI'] = vispubdata['DOI'].str.lower()

#we first add the new papers to vispubdata
new_papers = ieeexplore_vispub_df[~ieeexplore_vispub_df['DOI'].isin(vispubdata['DOI'])]

print(new_papers['Year'].unique()) #hopefully we're only adding papers from one new year here

vispubdata_new = pd.concat([vispubdata,new_papers],ignore_index = True)

#ok, I've learned that DOIs are not case-sensitive. Merde...
vispubdata_new["DOI"] = vispubdata_new["DOI"].str.lower()


#print(vispubdata_new.info())
vispubdata_new.to_csv("results/DEBUG_vispubdata_new.csv")


['2014' '2011' '2015' '2013' '2012' '2018' '2019' '2016' '2017' '2020'
 '2021' '2022' '2023' '2024']


### Copy over the download data

The IEEEXplore API provides information on how many times a paper has been downloaded. We copy this over here


In [64]:
download_df = pd.DataFrame({'DOI':ieeexplore_vispub_df['DOI'],downloads_column:ieeexplore_vispub_df[downloads_column]})
download_df["DOI"] = download_df["DOI"].str.lower()

for index,row in download_df.iterrows():
    downloads = row[downloads_column]
    doi = row['DOI']

    vispubdata_new.loc[vispubdata_new['DOI']== doi,downloads_column] = downloads

#check that now we have download counts updated
vispubdata_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Journal                     553 non-null    object
 1   Year                        553 non-null    object
 2   Title                       553 non-null    object
 3   DOI                         553 non-null    object
 4   Link                        553 non-null    object
 5   FirstPage                   553 non-null    object
 6   LastPage                    553 non-null    object
 7   PaperType                   553 non-null    object
 8   Abstract                    553 non-null    object
 9   AuthorNames-Deduped         553 non-null    object
 10  AuthorNames                 553 non-null    object
 11  AuthorAffiliation           553 non-null    object
 12  InternalReferences          553 non-null    object
 13  AuthorKeywords              553 non-null    object

### Add Awards

At this point we're not recording paper awards. This is a TODO...

### Add Replicability Stamp

In [66]:
replicablePapers = pd.read_csv("tvcg-dois-with-stamp.csv", keep_default_na=False)
replicablePapers["doi"] = replicablePapers["doi"].str.lower()
#replicablePapers.head()

stampMarker = "X"

for index,row in replicablePapers.iterrows():
    
    doi = row['doi']

    vispubdata_new.loc[vispubdata_new['DOI'] == doi,'GraphicsReplicabilityStamp'] = stampMarker


 

### Get CrossRef Data

The output from this operation is a file named  **citations_references_journals_vispubdata.csv**. You can jump over this section if this file is relatively recent since the next chunk of code takes a bit of time to run. The file will be read in by the subsequent steps.

In [71]:
my_etiquette = Etiquette('Vispubdata', '9.02', 'https://sites.google.com/site/vispubdata/home', youremail)
str(my_etiquette)

'Vispubdata/9.02 (https://sites.google.com/site/vispubdata/home; mailto:petra.isenberg@inria.fr) BasedOn: CrossrefAPI/1.5.0'

In [72]:
works = Works(etiquette=my_etiquette)

crossRefCitations = [ ]
crossRefPubsCited = [ ]
dois = [ ] 
referenced_papers = [ ]
referenced_papers_without_DOI = []

print("Starting to work on the citation counts")

for index,row in vispubdata_new.iterrows():
    doi = row['DOI']
    print(str(index) + " " + doi)
    
    paper = works.doi(doi)
    
    if (paper is None):
        print("CrossRef does not know: " + doi)
        dois.append(doi)
        crossRefCitations.append("")
        crossRefPubsCited.append("")
        referenced_papers_without_DOI.append("")
        referenced_papers.append("")
        continue
        
    isreferencedby = paper['is-referenced-by-count']
    
    if (isreferencedby is None):
        isreferencedby = ""
    
    references = paper['references-count']
    
    if (references is None):
        references = ""

    if 'reference' in paper:
        cited_papers = paper['reference']

        citedpapers = ""
        citedpapers_withoutDOI = ""

        if(cited_papers is None):
            citedpapers = ""
            print("No cited papers found for: " + doi)
        else:
            for ref in cited_papers:
                if "DOI" in ref:
                    cited_doi = ref['DOI'].lower().strip()
                    citedpapers =  citedpapers + ";" + cited_doi

                elif "article-title" in ref:
                    citedpapers_withoutDOI = citedpapers_withoutDOI + ":::" + ref['article-title'].strip()
                

        if len(citedpapers) > 0:
            citedpapers = citedpapers[1:]
        
        if len(citedpapers_withoutDOI) > 0:
            citedpapers_withoutDOI = citedpapers_withoutDOI[3:]
    else:
        print("No cited papers found for: " + doi)
        citedpapers_withoutDOI = ""
        citedpapers = ""
        #referenced_papers_without_DOI.append("")
        #referenced_papers.append("")
                
        
    crossRefCitations.append(isreferencedby)
    crossRefPubsCited.append(references)
    referenced_papers_without_DOI.append(citedpapers_withoutDOI)
    referenced_papers.append(citedpapers)

    dois.append(doi)


citationdf = pd.DataFrame({'DOI':dois,crossRefCitation_column:crossRefCitations,crossRefPubsCited_column:crossRefPubsCited,"citedPapers":referenced_papers,"citedPapersWithoutDOI":referenced_papers_without_DOI})
citationdf.to_csv("results/citations_references_journals_vispubdata.csv",index=False)
citationdf.head()


Starting to work on the citation counts
0 10.1109/tvcg.2013.254
1 10.1109/tvcg.2010.227
2 10.1109/mcg.2015.71
3 10.1109/tvcg.2010.93
4 10.1109/tvcg.2013.92
5 10.1109/tvcg.2011.109
6 10.1109/tvcg.2010.94
7 10.1109/tvcg.2012.118
8 10.1109/mcg.2017.377152546
9 10.1109/tvcg.2018.2843369
10 10.1109/tvcg.2014.2359887
11 10.1109/tvcg.2016.2520921
12 10.1109/tvcg.2016.2614803
13 10.1109/tvcg.2016.2615308
14 10.1109/tvcg.2016.2616404
15 10.1109/tvcg.2016.2549018
16 10.1109/tvcg.2016.2539960
17 10.1109/tvcg.2014.2350494
18 10.1109/tvcg.2016.2610422
19 10.1109/tvcg.2015.2440259
20 10.1109/tvcg.2015.2505305
21 10.1109/tvcg.2015.2392771
22 10.1109/tvcg.2016.2607204
23 10.1109/tvcg.2014.2329308
24 10.1109/tvcg.2014.2331979
25 10.1109/tvcg.2016.2535234
26 10.1109/tvcg.2014.2315995
27 10.1109/tvcg.2014.2330617
28 10.1109/tvcg.2015.2413786
29 10.1109/tvcg.2013.2297914
30 10.1109/mcg.2016.102
31 10.1109/tvcg.2017.2723393
32 10.1109/tvcg.2017.2701829
33 10.1109/tvcg.2017.2666146
34 10.1109/tvcg.2015.2511

,DOI,CitationCount_CrossRef,PubsCited_CrossRef,citedPapers,citedPapersWithoutDOI
0,10.1109/tvcg.2013.254,132,48,10.1111/j.1467-8306.1994.tb01869.x;10.1145/116...,Gephi: An Open Source Software for Exploring a...
1,10.1109/tvcg.2010.227,48,37,10.1016/s0167-2789(00)00199-8;10.1017/s0022112...,Pyramid Filters Based on Bilinear Interpolatio...
2,10.1109/mcg.2015.71,13,5,10.1038/nature05302;10.14714/cp63.165;10.1109/...,London-Edinburgh-London Info Sketches:::Visits...
3,10.1109/tvcg.2010.93,49,34,10.1109/vis.2005.35;10.1145/378583.378626;10.1...,Notes on the Topology of Vector Fields and Flo...
4,10.1109/tvcg.2013.92,28,42,10.1007/978-3-642-10520-3_9;10.1109/pacificvis...,Visualisation of Uncertainty Using the HSI Col...


### Integrate the Citation data

This part you shouldn't skip over.

In [73]:
citationdf = pd.read_csv("results/citations_references_journals_vispubdata.csv", keep_default_na=False)
citationdf.head()

,DOI,CitationCount_CrossRef,PubsCited_CrossRef,citedPapers,citedPapersWithoutDOI
0,10.1109/tvcg.2013.254,132,48,10.1111/j.1467-8306.1994.tb01869.x;10.1145/116...,Gephi: An Open Source Software for Exploring a...
1,10.1109/tvcg.2010.227,48,37,10.1016/s0167-2789(00)00199-8;10.1017/s0022112...,Pyramid Filters Based on Bilinear Interpolatio...
2,10.1109/mcg.2015.71,13,5,10.1038/nature05302;10.14714/cp63.165;10.1109/...,London-Edinburgh-London Info Sketches:::Visits...
3,10.1109/tvcg.2010.93,49,34,10.1109/vis.2005.35;10.1145/378583.378626;10.1...,Notes on the Topology of Vector Fields and Flo...
4,10.1109/tvcg.2013.92,28,42,10.1007/978-3-642-10520-3_9;10.1109/pacificvis...,Visualisation of Uncertainty Using the HSI Col...


In [74]:
#Here we copy the citation data over
#it would be much smarter to do this with a merge TODO later since I don't care for speed yet....

for index,row in citationdf.iterrows():
    
    doi = row['DOI']
    pubscited = row[crossRefPubsCited_column]
    citation = row[crossRefCitation_column]

    vispubdata_new.loc[vispubdata_new['DOI'] == doi,crossRefPubsCited_column] = pubscited
    vispubdata_new.loc[vispubdata_new['DOI'] == doi,crossRefCitation_column] = citation



vispubdata_new.to_csv("DEBUG_vispubdata_new.csv")

vispubdata_new.head()


,Journal,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount,CitationCount_CrossRef,PubsCited_CrossRef,Downloads_Xplore,Award,GraphicsReplicabilityStamp
0,TVCG,2014,GraphDiaries: Animated Transitions andTemporal...,10.1109/tvcg.2013.254,http://dx.doi.org/10.1109/TVCG.2013.254,740,754,J,"Identifying, tracking and understanding change...",,Benjamin Bach;Emmanuel Pietriga;Jean-Daniel Fe...,"INRIA, Saclay, France;INRIA, Saclay, France an...",,"Dynamic networks,graph visualization,,,,,,,,,,...",,132,48,1647,,
1,TVCG,2011,Hierarchical Line Integration,10.1109/tvcg.2010.227,http://dx.doi.org/10.1109/TVCG.2010.227,1148,1163,J,This paper presents an acceleration scheme for...,,Marcel Hlawatsch;Filip Sadlo;Daniel Weiskopf,Visualisierungsinstitut der Universität Stuttg...,,"Flow visualization,integral curves,,,,,,,,,,,h...",,48,37,614,,
2,CG&A,2015,Visualizing Personal Progress in Participatory...,10.1109/mcg.2015.71,http://dx.doi.org/10.1109/MCG.2015.71,73,81,MAG,This article explores the potential for creati...,,Jo Wood,"City University, London",,"computer graphics,personal visualization,,,,,,...",,13,5,1134,,
3,TVCG,2011,Stable Feature Flow Fields,10.1109/tvcg.2010.93,http://dx.doi.org/10.1109/TVCG.2010.93,770,780,J,Feature Flow Fields are a well-accepted approa...,,Tino Weinkauf;Holger Theisel;Allen Van Gelder;...,"Learning and Graphics Group, Courant Institute...",,"Flow visualization,feature extraction.",,49,34,775,,
4,TVCG,2013,Visualizing the Variability of Gradients in Un...,10.1109/tvcg.2013.92,http://dx.doi.org/10.1109/TVCG.2013.92,1948,1961,J,In uncertain scalar fields where data values v...,,Tobias Pfaffelmoser;Mihaela Mihai;Rüdiger West...,"Computer Graphics and Visualization Group, Tec...",,"Uncertainty visualization,gradient variability...",,28,42,581,,


### Getting the internal citations

What should the internal references represent? Any references to VIS papers? Or also references to other journal papers presented at VIS? Maybe let's go for all of VIS + journals. Although this is not what I'm currently doing for vispubdata where references are only to other VIS papers.

For this to work correctly the online version of vispubdata should have been updated - since I'm reading from there directly. Otherwise you'll have to load the latest local version.



In [85]:
# Google Sheet URL
vispubdata_main_sheet_url = "https://docs.google.com/spreadsheets/d/1xgoOPu28dQSSGPIp_HHQs0uvvcyLNdkMF9XtRajhhxU/gviz/tq?tqx=out:csv"

# Read the CSV into a DataFrame
vispubdata_main = pd.read_csv(vispubdata_main_sheet_url, keep_default_na=False)
vispubdata_main_DOIs = vispubdata_main['DOI'].str.lower().tolist()

vispubdata_journal_DOIs = vispubdata_new['DOI'].str.lower().tolist()

for index,row in citationdf.iterrows():
    
    doi = row['DOI'].lower()

    crossRefReferences = row['citedPapers']
    if pd.isna(crossRefReferences):
         #if crossref didn't return any references, for now we just continue. Later we can write code to check if it returned any titles of papers
         continue
    
    
    internalReferences_current = vispubdata_new.loc[vispubdata_new['DOI'] == doi,'InternalReferences']
    currentreflist = internalReferences_current #.tolist()[0].split(";")
    
    if not pd.isna(internalReferences_current).values[0]:
         currentreflist = currentreflist.tolist()[0].split(";")
    else:
         currentreflist = []
    
    #print("This many internal refs before: " + str(len(currentreflist)))
    refsbefore = len(currentreflist)
    currentreflist = [s.strip() for s in currentreflist]
    currentreflist = [s.lower() for s in currentreflist]
     
       
    crossRefList = crossRefReferences.split(";")
    crossRefList = [s.strip() for s in crossRefList]
    crossRefList = [s.lower() for s in crossRefList]

    for crossRefRef in crossRefList:
         if crossRefRef in currentreflist:
              #in this case the citation is already in our list
              #print("already found: " + crossRefRef)
              continue
         else:
              #the citation is not already on vispubdata. There are two reasons. Either the reference is not to a VIS paper, or it was forgotten
              #so let's check if the reference is a vis paper
              if (crossRefRef in vispubdata_journal_DOIs) or (crossRefRef in vispubdata_main_DOIs):
                   #yes, we've found a VIS paper. So now we need to add it to our list:
                   currentreflist.append(crossRefRef)

    #print("This many internal refs after: " + str(len(currentreflist)))
    #if len(currentreflist) > refsbefore:
    #     print("found something new")
    
    

    #print(crossRefReferences)
    
    vispubdata_new.loc[vispubdata_new['DOI'] == doi,'InternalReferences'] = ';'.join(currentreflist)[1:]

In [86]:
vispubdata_new.head()
#so we can double-check that things look good
vispubdata_new.to_csv("results/DEBUG_vispubdata_new.csv")

### Include deduped authors from DBLP

For the following code I expect that you already ran preparation 2 from all the way at the top of the document. That's the step where you extracted potential VIS authors from the DBLP

In [87]:
def find_doi_substring(text):
    # Define the regex pattern to find the DOI that starts with doi.org
    pattern = r'doi\.org/([^:]+?)(::|$)'
    
    # Search for the pattern in the given text
    match = re.search(pattern, text)
    
    # If a match is found, return the captured group (the DOI substring)
    if match:
        return match.group(1)
    else:
        return None


In [88]:
dblpauthors = pd.read_csv("../dblp-data-extraction/data/VIS-author-articles.csv",keep_default_na=False)


# Define the regex pattern to find the doi out of the list of electronic identifiers
dblpauthors['DOI'] = dblpauthors['ee'].apply(find_doi_substring)
dblpauthors['DOI'] = dblpauthors['DOI'].str.lower()

dblpauthors.head(20)

,title,author,year,ee,DOI
conf/chi/0002TVCGHDFSBM24,Exploring Holistic HMI Design for Automated Ve...,Haoyu Dong 0002::Tram Thi Minh Tran::Rutger Ve...,2024,https://doi.org/10.1145/3613905.3651086,10.1145/3613905.3651086
conf/ieeevast/HuMCCJY19,Visual Analysis of Multivariate Time Series of...,Yueqi Hu::Qi Ma::Yang Chen::Haidong Chen::Weiq...,2019,https://doi.org/10.1109/VAST47406.2019.8986910,10.1109/vast47406.2019.8986910
conf/ieeevast/Pabst12,BusinessForensics HQ: VAST Challenge Mini Chal...,Robert Pabst,2012,https://doi.org/10.1109/VAST.2012.6400526::htt...,10.1109/vast.2012.6400526
conf/ieeevast/Glassner09,Using stories to understand patterns.,Andrew S. Glassner,2009,https://doi.org/10.1109/VAST.2009.5334118,10.1109/vast.2009.5334118
conf/ieeevast/FarrugiaQ08,Cell phone Mini Challenge: Node-link animation...,Michael Farrugia::Aaron J. Quigley,2008,https://doi.org/10.1109/VAST.2008.4677393::htt...,10.1109/vast.2008.4677393
conf/ieeevast/TakedaKKOM12,Irregular Trend Finder: Visualization tool for...,Shinnosuke Takeda::Aimi Kobayashi::Hiroaki Kob...,2012,https://doi.org/10.1109/VAST.2012.6400504::htt...,10.1109/vast.2012.6400504
conf/ieeevast/MigutWBGSLW12,VAST Challenge 2012: Interactively finding ano...,Gosia Migut::Justin van Wees::Diederik Bakker:...,2012,https://doi.org/10.1109/VAST.2012.6400508::htt...,10.1109/vast.2012.6400508
conf/ieeevast/SavikhinME08,Applied visual analytics for economic decision...,Anya Savikhin::Ross Maciejewski::David S. Ebert,2008,https://doi.org/10.1109/VAST.2008.4677363::htt...,10.1109/vast.2008.4677363
conf/ieeevast/NunesRSKMLB14,An integrated visual analysis system for fusin...,Miguel Nunes::Benjamin Rowland::Matthias Schla...,2014,https://doi.org/10.1109/VAST.2014.7042481::htt...,10.1109/vast.2014.7042481
conf/ieeevast/BensonRAV15,Case study of dino fun world movement and comm...,Jordan Riley Benson::Rajiv Ramarajan::Nascif A...,2015,https://doi.org/10.1109/VAST.2015.7347652::htt...,10.1109/vast.2015.7347652


In [89]:
# let's prepare to copy things over

vispubdata_new['AuthorNames-Deduped'] = vispubdata_new['AuthorNames-Deduped'].astype(str)

dois_not_in_dblp = []


for index,row in vispubdata_new.iterrows():

    doi = row['DOI']
    #find the DOI in the dblpauthors
    
    #print(doi)
    found = dblpauthors['DOI'].eq(doi).any() 
    #print(found)
   
    if(found != False):

        dblp_rowindex = dblpauthors.index[dblpauthors['DOI'] == doi].tolist()[0]
        #print(dblp_rowindex)
        vispubdata_rowindex = index #pd.Index(vispubdata.DOI).get_loc(doi)
        #print(vispubdata_rowindex)
        
        vispubdata_deduped_authors = vispubdata_new.at[vispubdata_rowindex,'AuthorNames-Deduped']
        authors_on_dblp = dblpauthors.at[dblp_rowindex,'author']
        authors_on_dblp = authors_on_dblp.replace("::",";")
        

        if vispubdata_deduped_authors != authors_on_dblp:
            dblp_deduped_authors = authors_on_dblp
            print("DOI: " +dblpauthors.at[dblp_rowindex,'DOI'] + " " + vispubdata_new.at[vispubdata_rowindex,'DOI'])
            print("Existing authors do not match new authors:")
            print(" old: "+str(vispubdata_deduped_authors))
            print(" new: "+str(dblp_deduped_authors))
            vispubdata_new.at[vispubdata_rowindex,'AuthorNames-Deduped'] = dblp_deduped_authors
        #else:
            #print("Old authors match new authors")
    else:
        dois_not_in_dblp.append(doi)

print(len(dois_not_in_dblp))




DOI: 10.1109/mcg.2017.377152546 10.1109/mcg.2017.377152546
Existing authors do not match new authors:
 old: Lhaylla Crissaff;Louisa Wood Ruby;Samantha Deutch;R. Luke DuBois;Jean-Daniel Fekete;Juliana Freire;Claudio Silva
 new: Lhaylla Crissaff;Louisa Wood Ruby;Samantha Deutch;R. Luke DuBois;Jean-Daniel Fekete;Juliana Freire;Cláudio T. Silva
DOI: 10.1109/mcg.2016.102 10.1109/mcg.2016.102
Existing authors do not match new authors:
 old: Manuel Stein;Halldór Janetzko;Thorsten Breitkreutz;Daniel Seebacher;Tobias Schreck;Michael Grossniklaus;Iain D. Couzin;Daniel A. Keim
 new: Manuel Stein;Halldor Janetzko;Thorsten Breitkreutz;Daniel Seebacher;Tobias Schreck;Michael Grossniklaus;Iain D. Couzin;Daniel A. Keim
DOI: 10.1109/mcg.2016.101 10.1109/mcg.2016.101
Existing authors do not match new authors:
 old: Marcos Lage;Jorge Piazentin Ono;Daniel Cervone;Justin Chiang;Carlos Dietrich;Claudio T. Silva
 new: Marcos Lage;Jorge Piazentin Ono;Daniel Cervone;Justin Chiang;Carlos A. Dietrich;Cláudio T. 

In [90]:
print(dois_not_in_dblp)


#for those papers we just copy over their IEEEXplore author list
for doi in dois_not_in_dblp:
    vispubdata_rowindex = pd.Index(vispubdata_new.DOI).get_loc(doi)
    authors = vispubdata_new.at[vispubdata_rowindex,'AuthorNames']
    vispubdata_new.loc[vispubdata_rowindex, 'AuthorNames-Deduped'] = authors

['10.1109/tvcg.2023.3245609', '10.1109/tvcg.2023.3240003', '10.1109/tvcg.2023.3239909', '10.1109/tvcg.2023.3285210', '10.1109/tvcg.2023.3238821', '10.1109/tvcg.2023.3285546', '10.1109/tvcg.2022.3229023', '10.1109/tvcg.2023.3236380', '10.1109/tvcg.2023.3251344', '10.1109/tvcg.2022.3232959', '10.1109/tvcg.2022.3229953', '10.1109/tvcg.2023.3241581', '10.1109/tvcg.2022.3231716', '10.1109/tvcg.2023.3258440', '10.1109/tvcg.2023.3293121', '10.1109/tvcg.2023.3241596', '10.1109/tvcg.2023.3263856', '10.1109/tvcg.2023.3286392', '10.1109/tvcg.2022.3225554', '10.1109/tvcg.2023.3278304', '10.1109/tvcg.2022.3233548', '10.1109/tvcg.2023.3251950', '10.1109/tvcg.2023.3243676', '10.1109/tvcg.2023.3262039', '10.1109/tvcg.2023.3256376', '10.1109/tvcg.2023.3292391', '10.1109/tvcg.2022.3230445', '10.1109/tvcg.2023.3240356', '10.1109/tvcg.2023.3259341', '10.1109/tvcg.2023.3282364', '10.1109/tvcg.2023.3288356']


All articles above are not (yet) in DBLP. They may be early access articles.

## The Finale
We save the file to disk that can be copied over to vispubdata. However, at this point the Aminer citations are still missing. 


In [102]:
vispubdata_new.to_csv("results/vispubdata-update-journals.csv",index=False)

## Appendix
Now we generate a few files that are handy for data analysis about authors. This is not strictly necessary for the vispubdata update


In [97]:
deduped_authors = []

authors = vispubdata_new['AuthorNames-Deduped'].dropna()

for author in authors:
    authors = author.split(';')
    for a in authors:
        deduped_authors.append(a)
    

deduped_unique_authors = list(set(deduped_authors))

#print(deduped_unique_authors)    

pd.DataFrame({'Authors':deduped_unique_authors}).to_csv('results/Deduped-Authors-journals.csv',index=False)

In [98]:


#create the papers -> author names df
papersAuthorsTemp = pd.DataFrame({'Paper DOI':vispubdata_new['DOI'],'Author Names':vispubdata_new['AuthorNames-Deduped']})
papersAuthors = pd.DataFrame(papersAuthorsTemp['Author Names'].str.split(';').tolist(), index=papersAuthorsTemp['Paper DOI']).stack()
papersAuthors = papersAuthors.reset_index([0, 'Paper DOI'])
papersAuthors.columns = ['Paper DOI', 'Author Names']

papersAuthors.to_csv("results/DedupedAuthors-Papers-journals.csv",index=False)

papersAuthors.head()

,Paper DOI,Author Names
0,10.1109/tvcg.2013.254,Benjamin Bach
1,10.1109/tvcg.2013.254,Emmanuel Pietriga
2,10.1109/tvcg.2013.254,Jean-Daniel Fekete
3,10.1109/tvcg.2010.227,Marcel Hlawatsch
4,10.1109/tvcg.2010.227,Filip Sadlo


In [99]:
papersAuthorsPosition = papersAuthors
papersAuthorsPosition['PositionNumber'] = -1
papersAuthorsPosition['PositionCode'] = ''
papersAuthorsPosition['Affiliation'] = ''

papersAuthorsPosition.head()
lastDOI = ''
lastPosition = 0
lastindex = -1

#Warning, this assumes that the df was created in sorted order
for index, row in papersAuthorsPosition.iterrows():
    doi = row['Paper DOI']
    position = -1
    affiliation = ''
    
    allAffs = vispubdata_new.loc[vispubdata_new['DOI'] == doi,'AuthorAffiliation'].values[0]
    #print(allAffs)
    
    affList = allAffs.split(";")
    
    if doi != lastDOI:
        #we've reached a new paper
        position = 1
        positionCode = 'F'
        lastDOI = doi
        if len(affList) > 0:
            affiliation = affList[0]
        
        #if we're not at the very first row
        if lastindex != -1:
            #we need to update the positionCode of the last author
            papersAuthorsPosition.loc[lastindex,'PositionCode'] = 'L'
            #if it was a single author, we set them to 'F'
            if papersAuthorsPosition.loc[lastindex,'PositionNumber'] == 1:
                papersAuthorsPosition.loc[lastindex,'PositionCode'] = 'F'

    elif doi == lastDOI:
        #we're still at the same paper
        if len(affList) > lastPosition:
            affiliation = affList[lastPosition]
        position = lastPosition + 1
        positionCode = 'M'
        
        
    papersAuthorsPosition.loc[index,'PositionNumber'] = position
    papersAuthorsPosition.loc[index,'PositionCode'] = positionCode
    papersAuthorsPosition.loc[index,'Affiliation'] = affiliation
    lastindex = index
    lastPosition = position
        
        
papersAuthorsPosition.to_csv("results/DedupedAuthors-Papers-Position-Affiliation-journals.csv",index=False)

papersAuthorsPosition.head(10)


,Paper DOI,Author Names,PositionNumber,PositionCode,Affiliation
0,10.1109/tvcg.2013.254,Benjamin Bach,1,F,"INRIA, Saclay, France"
1,10.1109/tvcg.2013.254,Emmanuel Pietriga,2,M,"INRIA, Saclay, France and INRIA Chile-CIRIC, S..."
2,10.1109/tvcg.2013.254,Jean-Daniel Fekete,3,L,"INRIA, Saclay, France"
3,10.1109/tvcg.2010.227,Marcel Hlawatsch,1,F,Visualisierungsinstitut der Universität Stuttg...
4,10.1109/tvcg.2010.227,Filip Sadlo,2,M,Visualisierungsinstitut der Universität Stuttg...
5,10.1109/tvcg.2010.227,Daniel Weiskopf,3,L,Visualisierungsinstitut der Universität Stuttg...
6,10.1109/mcg.2015.71,Jo Wood,1,F,"City University, London"
7,10.1109/tvcg.2010.93,Tino Weinkauf,1,F,"Learning and Graphics Group, Courant Institute..."
8,10.1109/tvcg.2010.93,Holger Theisel,2,M,"AG Visual Computing, Fakultät für Informatik, ..."
9,10.1109/tvcg.2010.93,Allen Van Gelder,3,M,"University of California Santa Cruz, Santa Cru..."
